# Building Custom Tools.

At their core, tools are objects that consume some input, typically in the format of a string (text), and output some helpful information as a string.

In reality, they are little more than a simple function that we’d find in any code. The only difference is that tools take input from an LLM and feed their output to an LLM.



In [1]:
!pip install -qU langchain langchain_openai langchain_community transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


# Custom Tool to calculate Circumference.

**BaseTool**: We can think of the BaseTool as the required template for a LangChain tool.

We have two attributes that LangChain requires to recognize an object as a valid tool.
Those are the
1. name and
2. description
parameters.

**Description**: The description is a natural language description of the tool the LLM uses to decide whether it needs to use it. Tool descriptions should be very explicit on what they do, when to use them, and when not to use them.

In our description, we did not define when not to use the tool. That is because the LLM seemed capable of identifying when this tool is needed. Adding “when not to use it” to the description can help if a tool is overused.

Following this, we have two methods,
1. _run and
2. _arun.

When a tool is used, the _run method is called by default. The _arun method is called when a tool is to be used asynchronously

In [5]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')

In [6]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

llm = ChatOpenAI(
    temperature=0
)


memory = ConversationBufferWindowMemory(
    memory_key = 'chat_history',
    k = 5,
    return_messages=True
)

In [7]:
from langchain.tools import BaseTool
from math import pi
from typing import Union



```
Initialize Chain  ------>  Create Tool  ----->  Append to Tools  ----> Create Agent ----> Run Agent.
```

1. Build a chain.
2. Create a tool to call that chain.
3. Append all the tool to tools together.
4. Initialize the agent.
5. Run the agent.


In [50]:
# Creating Custom Tool

class CircumferenceTool(BaseTool):
  name = "Circumference Calculator"
  description = " Useful when you need to calculate circumference of a circle using the radius."

  def _run(self, radius: Union[int, float]):
    return float(radius)*2.0*pi

  def _arun(self, radius: Union[int, float]):
    raise NotImplementedError("This Tool Doesnot support async")

# Append Tools
tools = [CircumferenceTool()]

In [51]:
# Create Agent
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent = 'chat-conversational-react-description',
    tools=tools,
    llm = llm,
    memory=memory,
    max_iterations=3,
    early_stopping_method='generate',
    verbose=True
)

In [52]:
# Run agent
agent("circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference Calculator",
    "action_input": "7.81"
}
```
Observation: 49.071677249072565
Thought:```json
{
    "action": "Final Answer",
    "action_input": "49.071677249072565"
}
```

> Finished chain.


{'input': 'circumference of a circle that has a radius of 7.81mm',
 'chat_history': [HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.')],
 'output': '49.071677249072565'}

In [13]:
(7.81 * 2) * pi

49.071677249072565

### Existing Prompt

In [15]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

### We can overwrite the existing prompt.

In [30]:
sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""


In [31]:
new_prompt = agent.agent.create_prompt(
    system_message = sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [32]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, Assistant is terrible at maths. When provided with math question

# Tools with Multiple Parameters.

## Pythagoras Tool.

In [33]:
from typing import Optional
from math import sqrt, cos, sin

In [53]:
desc = (
    "Useful when you need to calculate the length of an hypotenuse " +
    "given one or two sides of a triangle and/or an angle (in degrees). " +
    "To use this tool, there must be atleast two of the following parameters:" +
    "['adjacent_side', 'opposite_side', 'angle']"
)

In [54]:
desc

"Useful when you need to calculate the length of an hypotenuse given one or two sides of a triangle and/or an angle (in degrees). To use this tool, there must be atleast two of the following parameters:['adjacent_side', 'opposite_side', 'angle']"

In [56]:
# Create Tool

class PythagorasTool(BaseTool):
  name = "Hypotenuse Calculator"
  description = desc

  def _run(
      self,
      adjacent_side: Optional[Union[int, float]] = None,
      opposite_side: Optional[Union[int, float]] = None,
      angle: Optional[Union[int, float]] = None
  ):
    # check for the values we have been given
    if adjacent_side and opposite_side:
        return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)

    elif adjacent_side and angle:
        return adjacent_side / cos(float(angle))

    elif opposite_side and angle:
        return opposite_side / sin(float(angle))

    else:
        return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."

  def _arun(self, query:str):
    raise NotImplementedError("This tool doesnot support async")


# Append Tools
tools.append(PythagorasTool())

In [58]:
tools

[CircumferenceTool(), PythagorasTool()]

In [63]:
# Create agent
agent = initialize_agent(
    agent = 'structured-chat-zero-shot-react-description',
    tools=tools,
    llm = llm,
    memory=memory,
    max_iterations=3,
    early_stopping_method='generate',
    verbose=True
)

In [64]:
# Run agent

agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...
Thought: This is a right triangle, so I can use the Hypotenuse Calculator to find the length of the hypotenuse by providing the lengths of the two sides.

Action:
```{
  "action": "Hypotenuse Calculator",
  "action_input": {
    "adjacent_side": 51,
    "opposite_side": 34
  }
}
```


Observation: 61.29437168288782
Thought:Great! The length of the hypotenuse in the triangle with sides of 51cm and 34cm is approximately 61.29cm.
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "The length of the hypotenuse is approximately 61.29cm."
}
```

> Finished chain.


{'input': 'If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?',
 'chat_history': [HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565')],
 'output': 'The length of the hypotenuse is approximately 61.29cm.'}

In [65]:
agent("circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
Thought: The user is asking for the circumference of a circle with a radius of 7.81mm. I can use the Circumference Calculator tool to find the answer.

Action:
```
{
  "action": "Circumference Calculator",
  "action_input": {
    "radius": 7.81
  }
}
```
Observation: 49.071677249072565
Thought:I have calculated the circumference of a circle with a radius of 7.81mm to be approximately 49.07mm.
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}
```

> Finished chain.


{'input': 'circumference of a circle that has a radius of 7.81mm',
 'chat_history': [HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='49.071677249072565'),
  HumanMessage(content='If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?'),
  AIMessage(content='The length of the hypotenuse is approximately 61.29cm.')],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}